In [21]:
import os 
import json
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [12]:
load_dotenv()
def langchain_approach():
    """Langchain - clean and simple"""
    llm = ChatOpenAI(
        model="gpt-4.1-mini",
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=os.getenv("OPENAI_API_BASE")
    )

    response = llm.invoke("Explain machine learning in one sentence")

    if response:
        print(f"Response: {response.content[:100]}...")
        return response.content 
    return None

if __name__ == "__main__":
    answer = langchain_approach()
    if answer:
        print("Full answer:")
        print(answer)


Response: Machine learning is a field of artificial intelligence where computers learn patterns from data to m...
Full answer:
Machine learning is a field of artificial intelligence where computers learn patterns from data to make predictions or decisions without being explicitly programmed.


In [17]:
template = PromptTemplate(
    input_variables=["topic", "style"],
    template="Explain {topic} in {style}"
)

print("\nTesting template with AI")
print("="*50)

llm_prompt = ChatOpenAI(
        model="gpt-4.1-mini",
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=os.getenv("OPENAI_API_BASE"),
        temperature=0.7
    )

if template and llm_prompt:
    test_prompt = template.format(
        topic = "artificial intelligence",
        style = "exactly 5 words"
    )

print(f"Sending to AI: {test_prompt}")

response = llm_prompt.invoke(test_prompt)
print(f"\nAI response: {response.content}")


Testing template with AI
Sending to AI: Explain artificial intelligence in exactly 5 words

AI response: Machines simulating human-like intelligence.


In [20]:
llm_output = ChatOpenAI(
        model="gpt-4.1-mini",
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=os.getenv("OPENAI_API_BASE"),
        temperature=0
    )
list_parser = CommaSeparatedListOutputParser()

list_prompt = PromptTemplate(
    template="List 3 benefits of {technology} (comma-separated):",
    input_variables=["technology"]
)

list_chain = list_prompt | llm_output | list_parser

if list_chain:
    result = list_chain.invoke({
        "technology": "cloud computing"
    })

    print(f"Input: List 3 benefits of cloud computing")
    print(f"Parsed Output: {result}")
    print(f"Type: {type(result)} - It's a Python list")
    print(f"Access items: result[0] = '{result[0]}")

Input: List 3 benefits of cloud computing
Parsed Output: ['Scalability', 'cost-efficiency', 'accessibility']
Type: <class 'list'> - It's a Python list
Access items: result[0] = 'Scalability


In [24]:
llm_json = ChatOpenAI(
        model="gpt-4.1-mini",
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=os.getenv("OPENAI_API_BASE"),
        temperature=0
    )
json_parser = JsonOutputParser()

json_prompt = PromptTemplate(
    template="""Analyze {technology} and respond with JSON containing:
    - benefits: array of 2 benefits
    - complexity: low/medium/high
    - use_case: one main use case

    Technology : {technology}

    {format_instructions}""",
    input_variables=["technology"],
    partial_variables={"format_instructions": json_parser.get_format_instructions()}
)

json_chain = json_prompt | llm_json | json_parser

if json_chain:
    result_1 = json_chain.invoke({
        "technology": "machine learning"
    })

    print(f"Input: Analyzing machine learning")

    try:
        parsed = result_1
        print(" Parsed Json output:")
        print(f" Benefits: {parsed.get('benefits',[])}")
        print(f" Complexity: {parsed.get('complexity', 'N/A')}")
        print(f" Use case: {parsed.get('use_case', 'N/A')}")
        print(f" Type: {type(parsed)} - It's a Python Dict")
    except (json.JSONDecodeError, TypeError, AttributeError):
        print(f"Parsing failed (rare with Jsonoutputparser): {result}")
        
print("\n Parser Magic:")
print(" List Parser: Text -> Python list")
print(" JSON parser: -> Python Dict")
print(" Direct data access: result_1[0], parsed['benefits]" )


Input: Analyzing machine learning
 Parsed Json output:
 Benefits: ['Ability to automatically learn and improve from data without explicit programming', 'Can handle and analyze large and complex datasets to uncover patterns and insights']
 Complexity: high
 Use case: Predictive analytics for customer behavior
 Type: <class 'dict'> - It's a Python Dict

 Parser Magic:
 List Parser: Text -> Python list
 JSON parser: -> Python Dict
 Direct data access: result_1[0], parsed['benefits]


In [27]:
llm_ana = ChatOpenAI(
        model="gpt-4.1-mini",
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=os.getenv("OPENAI_API_BASE"),
        temperature=0.3
    )

analysis_prompt = PromptTemplate(
    template="Analyze {technology} and provide pros and cons in 2-3 sentences",
    input_variables=["technology"]
)

str_parser = StrOutputParser()

analysis_chain = analysis_prompt | llm_ana | str_parser

if analysis_chain:
    result_2 = analysis_chain.invoke({
        "technology": "blockchain"
    })
    print(f"Input: Analyze Blockchain")
    print(f"Output: {result}")


Input: Analyze Blockchain
Output: ['Scalability', 'cost-efficiency', 'accessibility']


In [28]:
generator_prompt = PromptTemplate(
    template="List 3 use cases for {technology} (comma-separated):",
    input_variables=["technology"]
)

generator_parser = CommaSeparatedListOutputParser()

genrator_chain = generator_prompt | llm_ana | generator_parser

if genrator_chain:
    result_3 = genrator_chain.invoke({
        "technology": "blockchain"
    })
    print(f" Input: 'List use cases for blockchain'")
    print(f"Output: {result}")

test_tech = "artificial intelligence"
print(f"Technology: {test_tech}\n")

if analysis_chain and genrator_chain:
    analysis = analysis_chain.invoke({"technology": test_tech})
    print(f" Anlysis:\n {analysis}")

    use_cases = genrator_chain.invoke({"technology": test_tech})
    print(f"\nUse cases: ")
    for i, use_case in enumerate(use_cases,1):
        print(f"    {i}. {use_case}")

 Input: 'List use cases for blockchain'
Output: ['Scalability', 'cost-efficiency', 'accessibility']
Technology: artificial intelligence

 Anlysis:
 Artificial intelligence (AI) enhances efficiency and decision-making by automating complex tasks and analyzing vast data sets, driving innovation across industries. However, it also raises concerns about job displacement, ethical dilemmas, and potential biases embedded in algorithms. Balancing AI's benefits with responsible development is crucial for maximizing its positive impact.

Use cases: 
    1. image recognition
    2. natural language processing
    3. predictive analytics
